## Loads

In [1]:
import sys
import os
script_dir = os.getcwd()
root_dir = os.path.join(os.path.dirname(os.path.abspath(script_dir)))
sys.path.append(os.path.join(os.path.dirname(os.path.abspath(script_dir))))
os.chdir(root_dir)
os.getcwd()

'/home/helenajun/rag-llm-cancer-paper'

In [2]:
import json
import pandas as pd
import numpy as np
from utils.io import load_object, save_object
from collections import Counter
import requests
import random
import math
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
import re
from utils.entity_prediction import extract_entities, db_extract_entities, ner_predict_entities, clean_feature
import ast

In [3]:
#local db version
CACHE_FILE = "db_version_cache.json"

def get_remote_version():
    agents = requests.get('https://api.moalmanac.org/agents').json()
    return agents['service']['last_updated']

def get_local_version():
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE) as f:
            return json.load(f).get("version")
    return None

def save_local_version(version):
    with open(CACHE_FILE, "w") as f:
        json.dump({"version": version}, f)


#split ranges for testing on small batches
def split_ranges(max_int, num_ranges, samples=None, seed=42):
    random.seed(seed)
    step = math.ceil(max_int / num_ranges)
    ranges = [(i*step+1, min((i+1)*step, max_int)) for i in range(num_ranges)]
    
    if samples is None:
        return ranges
    
    # split samples evenly across ranges
    n_per_range = samples // num_ranges
    remainder = samples % num_ranges
    
    result = []
    for i, r in enumerate(ranges):
        count = n_per_range + (1 if i < remainder else 0)
        result.extend(random.randint(r[0], r[1]) for _ in range(count))
    
    return result


_VERSION=get_local_version()
_VERSION

'2025-09-04'

## Entity extraction using BioBert

In [4]:
biobert_dir = "context_retriever/biobert_ner"

In [ ]:
#download model from huggingface
model = AutoModelForTokenClassification.from_pretrained("judithrosell/BioBERT_BioNLP13CG_NER_new")
tokenizer = AutoTokenizer.from_pretrained("judithrosell/BioBERT_BioNLP13CG_NER_new")
id2label = model.config.id2label

#save locally
model.save_pretrained(biobert_dir)
tokenizer.save_pretrained(biobert_dir)

In [5]:
#load locally
model = AutoModelForTokenClassification.from_pretrained(biobert_dir)
tokenizer = AutoTokenizer.from_pretrained(biobert_dir)
id2label = model.config.id2label

### Context DB

In [6]:
#load structured context
with open(f"data/latest_db/indexes/text-embedding-3-small_structured_context__{_VERSION}.json", "r") as f:
    _CONTEXT = json.load(f)

for i, e in enumerate(_CONTEXT):
    print(e)
    if i == 5:
        break

If a patient with early invasive breast carcinoma cancer has her2-negative, er positive, one recommended therapy is tamoxifen + abemaciclib. therapy type: hormone therapy + targeted therapy. therapy strategy: estrogen receptor inhibition + cdk4/6 inhibition. indication: verzenio is a kinase inhibitor indicated in combination with endocrine therapy (tamoxifen or an aromatase inhibitor) for the adjuvant treatment of adult patients with hormone receptor (hr)-positive, human epidermal growth factor receptor 2 (her2)-negative, node positive, early breast cancer at high risk of recurrence. approval url: https://www.accessdata.fda.gov/drugsatfda_docs/label/2023/208716s010s011lbl.pdf
If a patient with early invasive breast carcinoma cancer has pr positive, her2-negative, one recommended therapy is tamoxifen + abemaciclib. therapy type: hormone therapy + targeted therapy. therapy strategy: estrogen receptor inhibition + cdk4/6 inhibition. indication: verzenio is a kinase inhibitor indicated in 

In [9]:
test_ind = split_ranges(len(_CONTEXT), 10, 20)
for i, e in enumerate(_CONTEXT):
    if i in test_ind:
        print(f"======={i}=======")
        print(e)
        print(extract_entities(e, model, tokenizer))

=======4=======
If a patient with advanced or metastatic invasive breast carcinoma cancer has pr positive, her2-negative, one recommended therapy is anastrozole + abemaciclib. therapy type: hormone therapy + targeted therapy. therapy strategy: aromatase inhibition + cdk4/6 inhibition. indication: verzenio is a kinase inhibitor indicated in combination with an aromatase inhibitor as initial endocrine-based therapy for the treatment of adult patients with hormone receptor (hr)-positive, human epidermal growth factor receptor 2 (her2)-negative advanced or metastatic breast cancer. approval url: https://www.accessdata.fda.gov/drugsatfda_docs/label/2023/208716s010s011lbl.pdf
{'cancer_type': ['breast carcinoma cancer', 'breast cancer'], 'biomarker': ['pr', 'cdk4', 'human epidermal growth factor receptor 2', 'her2', 'hormone receptor', 'aromatase', 'hr', None]}
=======15=======
If a patient with metastatic castration-resistant prostate adenocarcinoma cancer has bard1 oncogenic variants, one r

In [10]:
#run BioBert NER on all contexts
context_extracted_entities = []
for ctx in _CONTEXT:
    context_extracted_entities.append(extract_entities(ctx, model, tokenizer))
context_extracted_entities

[{'cancer_type': ['breast carcinoma cancer', 'early breast cancer'],
  'biomarker': ['aromatase',
   'cdk4',
   'her2',
   'hormone receptor',
   'estrogen receptor',
   'hr)',
   'er',
   'human epidermal growth factor receptor 2']},
 {'cancer_type': ['breast carcinoma cancer', 'breast cancer'],
  'biomarker': ['pr',
   'cdk4',
   'her2',
   'hormone receptor',
   'estrogen receptor',
   'aromatase',
   'hr',
   'human epidermal growth factor receptor 2']},
 {'cancer_type': ['breast carcinoma cancer', 'early breast cancer'],
  'biomarker': ['pr',
   'aromatase',
   'cdk4',
   'her2',
   'hormone receptor',
   'estrogen receptor',
   'hr)',
   'er',
   'human epidermal growth factor receptor 2']},
 {'cancer_type': ['breast carcinoma cancer', 'breast cancer'],
  'biomarker': ['aromatase',
   'cdk4',
   'human epidermal growth factor receptor 2',
   'her2',
   'hormone receptor',
   'er',
   'hr',
   None]},
 {'cancer_type': ['breast carcinoma cancer', 'breast cancer'],
  'biomarker': ['

In [11]:
no_cancer_idx = []
no_biomarker_idx = []
for i, e in enumerate(context_extracted_entities):
    if not e['cancer_type']:
        no_cancer_idx.append(i)
    if not e['biomarker']:
        no_biomarker_idx.append(i)

print(f"{len(no_cancer_idx)/len(_CONTEXT):.3f}%") #no cancer entity extracted %
print(f"{len(no_biomarker_idx)/len(_CONTEXT):.3f}%") #no biomarker entity extracted %

0.089%
0.002%


In [ ]:
#save
with open(f"context_retriever/entities/moalmanac_db_context_ner_entities__{_VERSION}.json", "w") as f:
    json.dump(context_extracted_entities, f)

### Cancer type and biomarker from DB

In [7]:
moalmanac_db = pd.read_csv(f"data/latest_db/moalmanac_core__{_VERSION}.csv")

In [11]:
#run BioBert NER on all DB entities
db_extracted_entities = []
for _, row in moalmanac_db.iterrows():
    db_extracted_entities.append(db_extract_entities(row))
db_extracted_entities

[{'cancer_type': ['early invasive breast carcinoma'],
  'biomarker': ['her2', 'er']},
 {'cancer_type': ['early invasive breast carcinoma'],
  'biomarker': ['pr', 'her2']},
 {'cancer_type': ['early invasive breast carcinoma'],
  'biomarker': ['pr', 'her2', 'er']},
 {'cancer_type': ['advanced or metastatic invasive breast carcinoma'],
  'biomarker': ['her2', 'er']},
 {'cancer_type': ['advanced or metastatic invasive breast carcinoma'],
  'biomarker': ['pr', 'her2']},
 {'cancer_type': ['advanced or metastatic invasive breast carcinoma'],
  'biomarker': ['pr', 'her2', 'er']},
 {'cancer_type': ['advanced or metastatic invasive breast carcinoma'],
  'biomarker': ['her2', 'er']},
 {'cancer_type': ['advanced or metastatic invasive breast carcinoma'],
  'biomarker': ['pr', 'her2']},
 {'cancer_type': ['advanced or metastatic invasive breast carcinoma'],
  'biomarker': ['pr', 'her2', 'er']},
 {'cancer_type': ['advanced or metastatic invasive breast carcinoma'],
  'biomarker': ['her2', 'er']},
 {'

In [12]:
#save
with open(f"context_retriever/entities/moalmanac_db_ner_entities__{_VERSION}.json", "w") as f:
    json.dump(db_extracted_entities, f)

### Synthetic query dataset

In [14]:
with open(f"data/latest_db/synthetic_prompts__{_VERSION}.json", "r") as f:
    synthetic_query = json.load(f)

In [15]:
#run BioBert NER on all synthetic queries
synthetic_query_extracted_entities = []
for query in synthetic_query:
    synthetic_query_extracted_entities.append(extract_entities(query, model, tokenizer))
synthetic_query_extracted_entities

[{'cancer_type': ['early invasive breast carcinoma cancer'],
  'biomarker': ['er', 'her2']},
 {'cancer_type': ['early invasive breast carcinoma cancer'],
  'biomarker': ['her2', 'pr']},
 {'cancer_type': ['early invasive breast carcinoma cancer'],
  'biomarker': ['er', 'her2', 'pr']},
 {'cancer_type': ['invasive breast carcinoma cancer'],
  'biomarker': ['er', 'her2']},
 {'cancer_type': ['invasive breast carcinoma cancer'],
  'biomarker': ['her2', 'pr']},
 {'cancer_type': ['invasive breast carcinoma cancer'],
  'biomarker': ['er', 'her2', 'pr']},
 {'cancer_type': ['invasive breast carcinoma cancer'],
  'biomarker': ['er', 'her2']},
 {'cancer_type': ['invasive breast carcinoma cancer'],
  'biomarker': ['her2', 'pr']},
 {'cancer_type': ['invasive breast carcinoma cancer'],
  'biomarker': ['er', 'her2', 'pr']},
 {'cancer_type': ['invasive breast carcinoma cancer'],
  'biomarker': ['er', 'her2']},
 {'cancer_type': ['invasive breast carcinoma cancer'],
  'biomarker': ['her2', 'pr']},
 {'canc

In [16]:
#save
with open(f"context_retriever/entities/synthetic_query_ner_entities__{_VERSION}.json", "w") as f:
    json.dump(synthetic_query_extracted_entities, f)

In [17]:
synthetic_query_extracted_entities[0]

{'cancer_type': ['early invasive breast carcinoma cancer'],
 'biomarker': ['er', 'her2']}

## Test entity matching

In [14]:
#test entities matching
from rapidfuzz import fuzz
from utils.entity_matching import match_entities

def check_list(input):
    if isinstance(input, list):
        input = input
    else:
        input = [input]
    return input

def match_entities(
    user_entities, 
    db_entities, 
    fuzzy_thres=70, 
    ):
    """
    Calculate score based on matching cancer types and biomarkers between user's query and the database
    
    Arguments:
        user_entities (dict): A dictionary with 'cancer_type' and 'biomarker' entities extracted using BioBERT.
        db_entities (list[dict]): A list of dictionaries with 'cancer_type' and 'biomarker' entities extracted from context database using BioBERT.
        fuzzy_thres (int): Threshold for fuzzy string similarity.
    """
    
    CANCER_FUZZY_SCORE = 1.0
    BIOMARKER_FUZZY_SCORE = 0.5

    user_cancer = check_list(user_entities.get('cancer_type', []))
    user_biomarker = check_list(user_entities.get('biomarker', []))

    match_score_all=[]
    
    #iterate over all db entities
    for idx, db_entity in enumerate(db_entities):
        score=0
        
        #append matching count
        for db_cancer in db_entity['cancer_type']: 
            db_cancer = check_list(db_cancer)
            if len(set(db_cancer) & set(user_cancer)) > 0:
                score += len(set(db_cancer) & set(user_cancer))
            elif any(fuzz.ratio(dbc, uc) > fuzzy_thres for uc in user_cancer for dbc in db_cancer):
                score += CANCER_FUZZY_SCORE
        
        for db_biomarker in db_entity['biomarker']:
            db_biomarker = check_list(db_biomarker)
            if len(set(db_biomarker) & set(user_biomarker)) > 0:
                score += len(set(db_biomarker) & set(user_biomarker))
            elif any(fuzz.ratio(dbb, ub) > fuzzy_thres for ub in user_biomarker for dbb in db_biomarker):
                score += BIOMARKER_FUZZY_SCORE

        if score > 0:
            match_score_all.append((idx, score, db_entity))

    #sort by score descending
    match_score_all.sort(key=lambda x: x[1], reverse=True)
    return match_score_all

with open(f"context_retriever/entities/moalmanac_db_ner_entities__{_VERSION}.json", "r") as f:
    _DB_ENTITY = json.load(f)
with open(f"context_retriever/entities/synthetic_query_ner_entities__{_VERSION}.json", "r") as f:
    _QUERY_ENTITY = json.load(f)

test_idx=0
print(_QUERY_ENTITY[test_idx])
matched_score_all=match_entities(_QUERY_ENTITY[test_idx], _DB_ENTITY, fuzzy_thres=70)
print(len(matched_score_all))
matched_score_all

{'cancer_type': ['early invasive breast carcinoma cancer'], 'biomarker': ['er', 'her2']}
161


[(0,
  3.0,
  {'cancer_type': ['early invasive breast carcinoma'],
   'biomarker': ['her2', 'er']}),
 (2,
  3.0,
  {'cancer_type': ['early invasive breast carcinoma'],
   'biomarker': ['pr', 'her2', 'er']}),
 (47,
  3.0,
  {'cancer_type': ['early invasive breast carcinoma'],
   'biomarker': ['pr', 'her2', 'er']}),
 (98,
  3.0,
  {'cancer_type': ['early invasive breast carcinoma'],
   'biomarker': ['her2', 'er']}),
 (148,
  3.0,
  {'cancer_type': ['invasive breast carcinoma'], 'biomarker': ['her2', 'er']}),
 (150,
  3.0,
  {'cancer_type': ['invasive breast carcinoma'],
   'biomarker': ['pr', 'her2', 'er']}),
 (151,
  3.0,
  {'cancer_type': ['invasive breast carcinoma'], 'biomarker': ['her2', 'er']}),
 (153,
  3.0,
  {'cancer_type': ['invasive breast carcinoma'],
   'biomarker': ['pr', 'her2', 'er']}),
 (225,
  3.0,
  {'cancer_type': ['metastatic invasive breast carcinoma'],
   'biomarker': ['her2', 'er']}),
 (227,
  3.0,
  {'cancer_type': ['metastatic invasive breast carcinoma'],
   'bi

In [15]:
matching_idx = [match[0] for match in matched_score_all]
matching_score = [match[1] for match in matched_score_all]

In [16]:
#load synthetic query
with open(f"data/latest_db/synthetic_prompts__{_VERSION}.json", "r") as f:
    synthetic_query=json.load(f)
    
#load groundtruth
synthetic_query_therapy_pair_dict=load_object("data/latest_db/synthetic_query_therapy_pair_dict.pkl")

#load retrieved context with high matching score
moalmanac_db = pd.read_csv(f"data/latest_db/moalmanac_core__{_VERSION}.csv")
moalmanac_db.therapy = moalmanac_db.therapy.apply(ast.literal_eval) 

In [18]:
print(synthetic_query[test_idx])
print(synthetic_query_therapy_pair_dict[synthetic_query[test_idx]])

if a patient with early invasive breast carcinoma cancer has her2-negative, er positive, what therapy is recommended?
[{'tamoxifen', 'abemaciclib'}, {'letrozole', 'abemaciclib'}, {'goserelin', 'anastrozole', 'ribociclib'}, {'goserelin', 'letrozole', 'ribociclib'}, {'anastrozole', 'abemaciclib'}, {'exemestane', 'abemaciclib'}]


In [19]:
subset = moalmanac_db[
    (moalmanac_db["modified_standardized_cancer"] == moalmanac_db["modified_standardized_cancer"].iloc[test_idx]) &
    (moalmanac_db["biomarker"] == moalmanac_db["biomarker"].iloc[test_idx])
]
subset

,statement_id,standardized_cancer,raw_cancer,modified_standardized_cancer,biomarker,therapy
0,0,Invasive Breast Carcinoma,early breast cancer,early invasive breast carcinoma,"['HER2-negative', 'ER positive']","[Tamoxifen, Abemaciclib]"
98,99,Invasive Breast Carcinoma,early breast cancer,early invasive breast carcinoma,"['HER2-negative', 'ER positive']","[Abemaciclib, Letrozole]"
577,579,Invasive Breast Carcinoma,early breast cancer,early invasive breast carcinoma,"['HER2-negative', 'ER positive']","[Goserelin, Anastrozole, Ribociclib]"
580,582,Invasive Breast Carcinoma,early breast cancer,early invasive breast carcinoma,"['HER2-negative', 'ER positive']","[Goserelin, Ribociclib, Letrozole]"
616,619,Invasive Breast Carcinoma,early breast cancer,early invasive breast carcinoma,"['HER2-negative', 'ER positive']","[Anastrozole, Abemaciclib]"
619,622,Invasive Breast Carcinoma,early breast cancer,early invasive breast carcinoma,"['HER2-negative', 'ER positive']","[Exemestane, Abemaciclib]"


In [20]:
Counter(matching_score).most_common()

[(2.0, 91), (1, 48), (3.0, 22)]

In [21]:
print(_QUERY_ENTITY[test_idx])

for i in subset.index:
    for rank, match in enumerate(matched_score_all):
        if match[0] == i:
            print(rank, match[1], match[2])

{'cancer_type': ['early invasive breast carcinoma cancer'], 'biomarker': ['er', 'her2']}
0 3.0 {'cancer_type': ['early invasive breast carcinoma'], 'biomarker': ['her2', 'er']}
3 3.0 {'cancer_type': ['early invasive breast carcinoma'], 'biomarker': ['her2', 'er']}
10 3.0 {'cancer_type': ['early invasive breast carcinoma'], 'biomarker': ['her2', 'er']}
12 3.0 {'cancer_type': ['early invasive breast carcinoma'], 'biomarker': ['her2', 'er']}
17 3.0 {'cancer_type': ['early invasive breast carcinoma'], 'biomarker': ['her2', 'er']}
19 3.0 {'cancer_type': ['early invasive breast carcinoma'], 'biomarker': ['her2', 'er']}


ok there are so many retrieved context with the same matching score...\
compared context-based entity extraction/matching to db field entity extraction/matching, which prioritizes ground-truth drugs way better \
now need to think how to integrate this into the pipeline